# Understanding Black-box Predictions via Influence functions

[Paper](http://proceedings.mlr.press/v70/koh17a/koh17a.pdf)

[Slides](https://danjacobellis.github.io/FTML/present_paper10.slides.html)


<script>
    document.querySelector('head').innerHTML += '<style>.slides { zoom: 1.75 !important; }</style>';
</script>

<center> <h1>
Understanding Black-box Predictions via Influence functions
</h1> </center>

## Background and caveats

* Our goal is to gain insights about how particular training points influence a model

* Loose definition of "black box" for this work
  * We **do** need to know the relationship between the loss and the model parameters
  * However, we don't need to know anything about the model architecture or how it works. Massive DNNs are fine.

The idea of examining the influence of particular training samples dates back to at least the 1970s.

For example, something that is used commonly in linear regression is Cook's distance.

Cook's distance is pretty simple. We want to know how an individual sample affects the prediction

To find the influence of the ith point, we train a model without that point, and observe the prediction. Here that would be this y with parentheses i.

We sum up the differences between the predictions with that point and without that point and then normalize by the number of parameters p and the mean squared error s squared.

That's cook's distance. It's quite useful. We can use it to find influential points for many reasons. We can go see if maybe that point was mislabeled. We can try to understand why that point was influential.

We can also use it to try to identify where we should sample the next time we go collect data.

## Cook's distance

* Dennis Cook, 1970: *Detection of influential observation in linear regression*
* "Overall summary statistics (e.g. $R^2$, $\beta$) ... present a distorted and misleading picture"
* **Cook's distance**: deviation of the model when point $i$ is excluded

$$D_i = \frac {1}{ps^2} \sum_{j=1}^{n}{(\hat y _j - \hat y_{j(i)}})^2$$

* Identify influential points
* Identify regions to sample when collecting subsequent data

A related quantity is the leverage. You can think of this as the degree to which the ith measured value influences the ith predicted value.

This is often used to detect outliers. A common heuristic is that if the leverage exceeds some multiple of p over n, the ratio of parameters to samples, then a point is an outlier.

Next, I'll define some different types of influence functions but just keep in mind that these functions are trying to do something similar to cook's distance or leverage, where we're looking at how particular point influence the model.

## Leverage 

* Also known as "Self-influence"
* Degree to which the $i$th measured value influences the $i$th predicted value
$$h_{ii} = \frac {\partial \hat y_i}{\partial y_i}$$
* $ 0 \leq h_{ii} \leq 1$
* Common heuristic: $x_i$ is an outlier if $h_{ii} > \frac {2p}{n}$

Let's start with the first type of influence function. Suppose you train a model. You minimize the empirical risk, resulting in some estimate of the best model parameters theta hat.

This theta hat is a function of the data distribution F. If we had a different set of data that we trained on that came from a different distribution, we would have gotten different estimates for our parameters.

The influence function is going to examine how a small perturbation in F affects theta hat.

Lets define F prime to be some new distribution that is almost the same as the original, except we've added some mass to a location in the sample space z.

The first type of influence function which we denote I theta, is like a derivative. It's the difference in model parameters that we get when changing the distribution in the limit that that change is very small.

If we're being very rigorous this is not a normal derivative but a functional one. But I think it's helpful to think about it the same way you think of any other derivative, as a rate of change.

So that's the first type of influence function, the rate of change of the model parameters when we add mass to some point in the distribution z.


## Influence function

* Consider a model parameter $\theta$ and our estimate of the best value after training $\hat \theta $
* $\hat \theta$ is a function of the data distribution $F$
* The influence function describes the effect on $\hat \theta$ when we make a small perturbation to the data
* In particular, we will add at location $z$ in the distribution $F$ an infinitesimal mass $\epsilon \delta_z$, resulting in a new distribution $F^\prime$

$$F^\prime = F(1-\epsilon) + \epsilon \delta_z$$

* The influence function is the derivative${}^\dagger$ of a model parameter $\hat \theta$ with respect to the distribution $F$. 

$$\mathcal I_{\hat \theta}(z) = \frac{d \hat \theta}{d F}^\dagger =  \lim_{\epsilon \to 0} \left[ \frac{ \hat \theta(F^\prime) - \hat \theta (F) }{\epsilon}\right]$$

$\dagger$ The derivative here is a functional derivative. In particular it is a Gateaux derivative.

The second type of influence function IL is similar, except instead of looking at the model parameters, we're looking at the loss function.

So, this is the rate of change in the loss function as we add mass to some location z.

## Influence function

* We can define a second type of influence function that describes the effect on the loss when we make a small perturbation to the data

$$\mathcal I_{L}(z) = \frac{d L}{d F} =  \lim_{\epsilon \to 0} \left[ \frac{ \hat L(F^\prime) - L (F) }{\epsilon}\right]$$

* A model is trained resulting in parameters $\hat \theta$. We test it on a point $z_{test}$. $\mathcal I_{L}(z,z_{test})$ is the rate at which the loss on $z_{test}$  increases as we make the training point $z$ more prevalent

Finally, we have this third type of influence function I pert

Instead of just adding a mass to some point in the distribution, we're also going to remove mass from some other point.

So this is basically just a difference between to influence functions of the normal type.

## Influence Function

* We can also define influence functions that describe the effect of 'moving' a training point

$$\mathcal I_{pert} = \mathcal I(z_2) -  \mathcal I(z_1)$$ 

So how do we actually calculate these influence functions?

Well the simplest approach is to just do something like we did for cook's distance and just train the model with all points except one point located at z. That will give us the influence function at z.

Clearly, this can get expensive quickly, so we'd like to have a more efficient method.

We could have tried to calculate the gradient of the model parameters or the loss function with respect to the data for example.

However, the authors go one step further. Why stop at the first derivative when we can use the second derivative? So that's what they do.

## Empirical influence function

### Expensive approach: Leave one out
* Train the model with all data, including some sample $z$, resulting in $\hat \theta$
* To estimate $\mathcal I_{\hat \theta}(z)$, train the model without the point $z$, resulting in $\hat \theta _ {(z)}$

We train the model with all of the data, resulting in the estimate of the optimal parameter values theta hat.

Starting from there, we locally approximate the loss function as a quadratic.

When we do that we get this result for the influence function.

We have the gradient of the loss function multiplied by the inverse of the hessian.

The hessian is just the matrix of second derivatives.

## Empirical influence function

### Cheaper approach: Locally approximate shape of loss function
* Start with a model trained on all of the data, including $z,$ resulting in $\hat \theta$
* Use a quadratic approximation of the loss function to estimate the effect of "upweighting" a sample $z$.

$$\mathcal I_{\hat \theta}(z) \approx -H_{\hat \theta} ^{-1} \nabla L$$

* $\nabla L$ is the gradient. $H$ is the *Hessian,* the matrix of all partial second derivatives.

* $-H_{\hat \theta} ^{-1} \nabla L$ is the direction you would move in while optimizing a function using Newton's method.

## Quadratic approximation

### Issue #1: Does the Hessian even exist?

* We have to pick a nice loss function so that the Hessian is guaranteed to exist.

  * Mean absolute error can be replaced with $\log \cosh (s)$. 
  * Hinge with smooth version $t \log {\left(1+\exp{\left(\frac{1-s}{t}\right)}\right)}.$

## Quadratic approximation

### Issue #2: Computing and storing the Hessian

* It is ***very*** expensive to compute the Hessian. Even just storing the Hessian for a large neural network might be impossible.

* The "Pearlmutter trick" allows us to calculate a matrix-vector product with the hessian $\mathbf {Hv}$ *exactly* with about as much computation as a single gradient evaluation.

* There are variations of how to do this. Some are exact some are not with tradeoffs in complexity.

* FOSS libraries rolled out in 2018 that allow you plug in your model and efficiently compute the influence using this method.

  * Popular one is [Darkon](https://github.com/darkonhub/darkon)
  
  * [Example notebook]( https://github.com/darkonhub/darkon-examples/blob/master/cifar10-resnet/influence_cifar10_resnet.ipynb)

## Experiments and Applications

## How well does the quadratic approximation work?

![](img/performance_1.png)

<p style="text-align:center;">
<img src="_images/performance_1.png" width=650 height=650 class="center">
</p>

## Do differentiable versions of the loss function work? 

![](img/hinge_1.png)

<p style="text-align:center;">
<img src="_images/hinge_1.png" width=750 height=750 class="center">
</p>

## Find helpful and harmful training samples

* Other methods exist to find relevant training samples.
  * For example, we can look at the nearest neighbor(s)
  
* The influence function can tell us if a training sample either helps or hurts when evaluating a particular test sample.
  * If $\mathcal I_{L}(z,z_{test})$ is positive, then the training point $z$ is harmful
  * If $\mathcal I_{L}(z,z_{test})$ is negative, then the training point $z$ is helpful

![](img/harmful_1.png)

<p style="text-align:center;">
<img src="_images/harmful_1.png" width=450 height=450 class="center">
</p>

## Identify overfitting and compare models

* Compare $\mathcal I_{L}(z,z_{test})$ with the euclidian distance

* If the model is overfitting, then all of the most influential training points will be neighbors.

* If the model generalizes, then the influential points will be spread out, not just overfit to neighboring points.

![](img/influence_1.png)

<p style="text-align:center;">
<img src="_images/influence_1.png" width=650 height=650 class="center">
</p>

## Debugging mismatch in distributions

* Consider a point the sample space $z$ and the surrounding region
* The training data may be very dense or very sparse in this region
* When deployed, the density may be quite different
* $\mathcal I_{\hat \theta, pert}(z)$ tells us exactly how we should update the model to account for this

## Debugging mislabeled examples and identifying places to collect new data

* Checking the top 5% highest magnitude influence points gives roughly the same performance as checking 60% of the training data exhaustively

![](img/mislabeled_1.png)

<p style="text-align:center;">
<img src="_images/mislabeled_1.png" width=250 height=250 class="center">
</p>

## Constructing and identifying adversarial examples

* Find an example that is difficult or ambiguous to start with
* At this point, find the gradient of the loss with respect to the data $\nabla _x L$
* Training on the point $x + \mu \nabla _x L$ will massively increase the error rate on any data near $x$
* The magnitude of $\mathcal I_{pert}(z)$ tells us how vulnerable a model is to an attack near $z$

![](img/adversarial_1.png)

<p style="text-align:center;">
<img src="_images/adversarial_1.png" width=750 height=750 class="center">
</p>

## Thank you!